In [5]:
!pip install -U https://s3-us-west-2.amazonaws.com/ray-wheels/latest/ray-3.0.0.dev0-cp38-cp38-manylinux2014_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 MB 32.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 54.3 MB/s eta 0:00:00:00:01
  Using cached virtualenv-20.15.1-py2.py3-none-any.whl (10.1 MB)
  Using cached distlib-0.3.5-py2.py3-none-any.whl (466 kB)
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


In [6]:
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


In [3]:
# Also available at s3://waleed-movies
#TODO: Read from s3 instead. Original files can be found at 
# https://www.kaggle.com/datasets/dadajonjurakuziev/movieposter
# Delete the ._*.jpg files 

files_dir = "/home/ec2-user/SageMaker/movie-recs/movie-posters"

import numpy as np
import pandas as pd
from io import BytesIO
from PIL import Image
import json
import ray
from ray.air.util.tensor_extensions.pandas import TensorArray
from ray.train.torch import to_air_checkpoint, TorchPredictor
from ray.train.batch_predictor import BatchPredictor

from torchvision import transforms
from torchvision.models.detection.ssd import ssd300_vgg16

In [5]:
ray.init(_system_config={
        "object_spilling_config": json.dumps(
            {"type": "filesystem", "params": {"directory_path": "/home/ec2-user/SageMaker/object-spill"}},
        )
    },
)

Usage stats collection is enabled by default for nightly wheels. To disable this, run the following command: `ray disable-usage-stats` before starting Ray. See https://docs.ray.io/en/master/cluster/usage-stats.html for more details.


Python version:,3.8.12
Ray version:,3.0.0.dev0


In [ ]:
def convert_to_pandas(byte_item_list):
    preprocess = transforms.Compose([
        transforms.Resize(300),
        transforms.CenterCrop((300,600)),
        transforms.ToTensor(),
    ])

    images = [Image.open(BytesIO(byte_item)).convert('RGB') for byte_item in byte_item_list]
    images = [preprocess(image) for image in images]
    images = [np.array(image) for image in images]

    return pd.DataFrame({"image": TensorArray(images)})

# What is parallelism magic number?
dataset = ray.data.read_binary_files(paths=files_dir, parallelism=2000)
dataset = dataset.map_batches(convert_to_pandas)

#TODO: show a visualization of one of these images

Read->Map_Batches:   0%|          | 0/2000 [00:00<?, ?it/s](_map_block_nosplit pid=3412) 2022-07-20 00:00:20,085	INFO worker.py:754 -- Task failed with retryable exception: TaskID(e0dc174c83599034ffffffffffffffffffffffff01000000).
(_map_block_nosplit pid=3412) Traceback (most recent call last):
(_map_block_nosplit pid=3412)   File "python/ray/_raylet.pyx", line 670, in ray._raylet.execute_task
(_map_block_nosplit pid=3412)   File "python/ray/_raylet.pyx", line 674, in ray._raylet.execute_task
(_map_block_nosplit pid=3412)   File "/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/ray/data/_internal/compute.py", line 441, in _map_block_nosplit
(_map_block_nosplit pid=3412)     for new_block in block_fn(block, *fn_args, **fn_kwargs):
(_map_block_nosplit pid=3412)   File "/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/ray/data/_internal/plan.py", line 605, in block_fn
(_map_block_nosplit pid=3412)     for tmp2 in block_fn2(tmp1, *self_fn_args, **s

In [ ]:
model = ssd300_vgg16(pretrained=True)

In [ ]:
# TODO: to_air_checkpoint should accept a model or a model state dict
ckpt = to_air_checkpoint(model=model)

predictor = BatchPredictor.from_checkpoint(ckpt, TorchPredictor)
results = predictor.predict(dataset, num_gpus_per_worker=1)

#TODO: visualize the images with the labels. 



NameError: name 'to_air_checkpoint' is not defined